In [1]:
import pyrealsense2 as rs
print("pyrealsense2 is installed correctly!")


pyrealsense2 is installed correctly!


In [3]:
!pip install ultralytics

In [5]:
import cv2
import numpy as np
import pyrealsense2 as rs
from ultralytics import YOLO
 
# Load the YOLOv8 model
model = YOLO('assignment_3_model_yolo11s.pt')
 
# Export the model to ONNX format
# model.export(format='onnx')
 
# Set up the RealSense D455 camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)
 
# Set the depth scale
depth_scale = 0.0010000000474974513
 
# Create spatial, temporal, and disparity filters
spatial = rs.spatial_filter()
temporal = rs.temporal_filter()
disparity_transform = rs.disparity_transform(True)  # Enable disparity transform
 
# Main loop
try:
    while True:
        # Get the latest frame from the camera
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()
 
        if not color_frame or not depth_frame:
            continue
 
        # Convert the frames to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())
 
        # Apply filters to depth image
        depth_frame = spatial.process(depth_frame)
        depth_frame = temporal.process(depth_frame)
        depth_frame = disparity_transform.process(depth_frame)  # Convert to disparity map
        depth_image = np.asanyarray(depth_frame.get_data())
 
        # Convert the depth image to meters
        depth_image = depth_image * depth_scale
 
        # Detect objects using YOLOv8
        results = model(color_image)
 
        # Process the results
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                confidence = box.conf[0].cpu().numpy()
                class_id = box.cls[0].cpu().numpy()
 
                if confidence < 0.5:
                    continue  # Skip detections with low confidence
 
                # Calculate the distance to the object
                object_depth = np.median(depth_image[y1:y2, x1:x2])
 
                # Get class name from YOLOv8 model
                class_name = model.names[int(class_id)]
 
                # Combine class name and distance in the label
                label = f"{class_name} {object_depth:.2f}m"
 
                # Draw bounding box
                cv2.rectangle(color_image, (x1, y1), (x2, y2), (252, 119, 30), 2)
 
               
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(color_image, (x1, y1 - 25), (x1 + w, y1), (252, 119, 30), -1)
                cv2.putText(color_image, label, (x1, y1 - 7), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (255, 255, 255), 2)
 
                # Print to console
                print(f"{class_name}: {object_depth:.2f}m")
 
        # Show the color image
        cv2.imshow("Color Image", color_image)
 
        # Show the disparity map
        disparity_map = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        cv2.imshow("Disparity Map", depth_image)
 
        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Stop the pipeline
    pipeline.stop()
    cv2.destroyAllWindows()


0: 480x640 1 AirPod, 1 Apple, 2 CardHolders, 1 Orange, 1 Watch, 419.4ms
Speed: 6.1ms preprocess, 419.4ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)
Apple: 1158108.76m
CardHolder: 0.00m
Orange: 1151138.50m

0: 480x640 1 AirPod, 2 CardHolders, 1 Orange, 1 StickyNote, 1 Watch, 382.2ms
Speed: 4.4ms preprocess, 382.2ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)
Orange: 1158429.75m
AirPod: 1161611.04m
CardHolder: 0.00m

0: 480x640 1 AirPod, 2 CardHolders, 1 Orange, 1 StickyNote, 402.7ms
Speed: 3.9ms preprocess, 402.7ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)
Orange: 1158737.30m
AirPod: 1161739.24m
CardHolder: 0.00m

0: 480x640 1 AirPod, 2 CardHolders, 2 Oranges, 1 StickyNote, 1 Watch, 329.9ms
Speed: 4.1ms preprocess, 329.9ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Orange: 1158967.09m
AirPod: 1161936.61m
CardHolder: 0.00m

0: 480x640 1 AirPod, 1 CardHolder, 2 Oranges, 1 StickyNote, 1 Watch, 395.1ms


In [ ]:
import cv2
import numpy as np
import pyrealsense2 as rs
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('assignment_3_model_yolo11s.pt')

# Set up RealSense D455 pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)

# Depth scale (RealSense provides depth in mm)
depth_scale = 0.001  # Convert to meters

# Create filters for cleaner depth
spatial = rs.spatial_filter()
temporal = rs.temporal_filter()

try:
    while True:
        # Capture frames
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()

        if not color_frame or not depth_frame:
            continue

        # Apply filters to depth
        depth_frame = spatial.process(depth_frame)
        depth_frame = temporal.process(depth_frame)

        # Convert to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float32) * depth_scale

        # Run YOLO detection
        results = model(color_image)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                confidence = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())

                if confidence < 0.5:
                    continue

                # Get depth from object region
                roi = depth_image[y1:y2, x1:x2]
                valid_depths = roi[(roi > 0.1) & (roi < 3.0)]  # Filter out invalid values

                object_depth = np.median(valid_depths) if valid_depths.size > 0 else 0.0

                # Get class name
                class_name = model.names[class_id]
                label = f"{class_name} {object_depth:.2f}m"

                # Draw bounding box and label
                cv2.rectangle(color_image, (x1, y1), (x2, y2), (252, 119, 30), 2)
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(color_image, (x1, y1 - 25), (x1 + w, y1), (252, 119, 30), -1)
                cv2.putText(color_image, label, (x1, y1 - 7), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (255, 255, 255), 2)

                # Print to console
                print(f"{class_name}: {object_depth:.2f}m")

        # Show result
        cv2.imshow("Color Image", color_image)

        # Optional: Show depth map (scaled for display)
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(depth_image, alpha=100), cv2.COLORMAP_JET
        )
        cv2.imshow("Depth Map", depth_colormap)

        # Quit on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()



0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 StickyNote, 1 Watch, 333.5ms
Speed: 7.7ms preprocess, 333.5ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
StickyNote: 0.33m
AirPod: 0.48m

0: 480x640 1 AirPod, 2 CardHolders, 1 Key, 1 Mobile, 1 Mouse, 1 Orange, 2 StickyNotes, 2 Watchs, 189.7ms
Speed: 1.9ms preprocess, 189.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Mobile: 0.34m
Orange: 0.50m
CardHolder: 0.41m
Mouse: 0.33m
AirPod: 0.48m
StickyNote: 0.54m

0: 480x640 1 AirPod, 2 CardHolders, 1 Mobile, 1 Mouse, 1 Orange, 1 StickyNote, 2 Watchs, 158.8ms
Speed: 1.4ms preprocess, 158.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
Orange: 0.50m
CardHolder: 0.41m
Mobile: 0.35m
Mouse: 0.33m
AirPod: 0.44m
StickyNote: 0.53m

0: 480x640 2 AirPods, 2 CardHolders, 1 Key, 1 Mobile, 1 Mouse, 1 Orange, 1 StickyNote, 1 Watch, 150.4ms
Speed: 1.8ms preprocess, 150.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
Mobile: 0

In [ ]:
#Base code

In [ ]:
import cv2
import imageio
import numpy as np
import pyrealsense2 as rs
import os
import shutil

pipe = rs.pipeline()
cfg = rs.config()

cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipe.start(cfg)

while True:
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha = 0.5), cv2.COLORMAP_JET)
    
    cv2.imshow('rgb', color_image)
    cv2.imshow('depth', depth_cm)
    if cv2.waitKey(1) == ord('q'):
        break

pipe.stop()

In [ ]:
#Last updated code

In [ ]:
import cv2
import imageio
import numpy as np
import pyrealsense2 as rs
import os
import shutil

pipe = rs.pipeline()
cfg = rs.config()

cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipe.start(cfg)

# Define the directory to store images
output_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2"

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

frame_count = 0

while True:
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha = 0.5), cv2.COLORMAP_JET)
    
    cv2.imshow('rgb', color_image)
    cv2.imshow('depth', depth_cm)





    # Increment frame counter
    frame_count += 1

    key = cv2.waitKey(1) & 0xFF
    if key == 32:
        # Save images to directory
        rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
        depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")
        
        # Save RGB and depth images
        cv2.imwrite(rgb_filename, color_image)
        cv2.imwrite(depth_filename, depth_cm)      
    elif key == 27:
        break

pipe.stop()

In [ ]:
#for each folder (final update) 

In [ ]:
import cv2
import numpy as np
import pyrealsense2 as rs
import os

# Define the base directory to store images
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2"

# Get user input for scenario, place, object, and distance
scenario_num = input("Enter Scenario (1, 2, 3): ")
place_num = input("Enter Place (1, 2, 3): ")
object_num = input("Enter Object (1-10): ")
distance_val = input("Enter Distance (X, Y, Z): ")

# Create directory path dynamically based on input
output_dir = os.path.join(
    base_dir, f"Scenario_{scenario_num}", f"Place_{place_num}", f"Object_{object_num}", f"Distance_{distance_val}"
)

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start pipeline
pipe.start(cfg)

# Frame counter and sequence count
frame_count = 0
frames_to_capture = 33  # Minimum frames per sequence

print(f"Saving images to: {output_dir}")
print("Press [SPACE] to capture frame or [ESC] to exit...")

while True:
    # Get frames
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap to depth image
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    # Show RGB and depth frames
    cv2.imshow('RGB', color_image)
    cv2.imshow('Depth', depth_cm)

    # Key press options
    key = cv2.waitKey(1) & 0xFF

    if key == 32:  # Spacebar to capture frames
        if frame_count < frames_to_capture:
            # Save images to directory
            rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
            depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")

            # Save RGB and depth images
            cv2.imwrite(rgb_filename, color_image)
            cv2.imwrite(depth_filename, depth_cm)

            frame_count += 1
            print(f"Captured frame {frame_count}/{frames_to_capture}...")
        else:
            print("Frame limit reached for this sequence.")
    
    elif key == 27:  # ESC to exit
        print("Exiting and stopping recording...")
        break

# Stop pipeline and release resources
pipe.stop()
cv2.destroyAllWindows()

print(f"Captured {frame_count}/{frames_to_capture} frames successfully. Images saved in '{output_dir}'.")


In [ ]:
#without fixed amount

In [ ]:
import cv2
import imageio
import numpy as np
import pyrealsense2 as rs
import os
from datetime import datetime

# Define the directory to store images
output_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images"

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start pipeline
pipe.start(cfg)

# Frame counter for unique filenames
frame_count = 0

while True:
    # Get frames
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap to depth image
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    # Show RGB and depth frames
    cv2.imshow('RGB', color_image)
    cv2.imshow('Depth', depth_cm)

    # Save images to directory
    rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
    depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")

    # Save RGB and depth images
    cv2.imwrite(rgb_filename, color_image)
    cv2.imwrite(depth_filename, depth_cm)

    # Increment frame counter
    frame_count += 1

    # Break if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop pipeline and release resources
pipe.stop()
cv2.destroyAllWindows()

print(f"Images saved in '{output_dir}' directory.")


In [ ]:
#For 100 frames

In [ ]:
import cv2
import numpy as np
import pyrealsense2 as rs
import os

# Define the directory to store images
output_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images"

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start pipeline
pipe.start(cfg)

# Frame counter for unique filenames
frame_count = 0
max_frames = 100  # Capture 100 frames and stop

while frame_count < max_frames:
    # Get frames
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Check if frames are valid
    if not depth_frame or not color_frame:
        continue

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap to depth image
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    # Show RGB and depth frames
    cv2.imshow('RGB', color_image)
    cv2.imshow('Depth', depth_cm)

    # Save images to directory
    rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
    depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")

    # Save RGB and depth images
    cv2.imwrite(rgb_filename, color_image)
    cv2.imwrite(depth_filename, depth_cm)

    # Increment frame counter
    frame_count += 1

    # Break if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        print("Stopping recording...")
        break

# Stop pipeline and release resources
pipe.stop()
cv2.destroyAllWindows()

print(f"Captured {frame_count} frames. Images saved in '{output_dir}' directory.")


In [ ]:
# Import required libraries
import cv2
import numpy as np

# Load the image
image_path = 'F:/1 KFUPM/3rd Semester/Computer Vision/Assignment 3/captured_images2/depth_frame_0119.jpg'  # Replace with your image path
image = cv2.imread(image_path)


In [ ]:
import cv2
import numpy as np

# Load the depth image
depth_image_path = 'path_to_your_depth_image.png'
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

# Define camera parameters
focal_length = 800  # Example value in pixels (check your camera specs)
baseline = 0.1  # Example value in meters (distance between stereo cameras)

# Pixel coordinates to calculate distance for (example: center pixel)
pixel_x, pixel_y = 320, 240  # Modify based on your target pixel

# Get the depth value at the pixel
depth_value = depth_image[pixel_y, pixel_x]

# Check if depth value is valid
if depth_value == 0 or np.isnan(depth_value):
    print("Invalid depth value at this pixel.")
else:
    # Calculate real-world distance (monocular depth case)
    real_world_distance = (depth_value * baseline) / focal_length

    print(f"Distance at pixel ({pixel_x}, {pixel_y}) is {real_world_distance:.2f} meters.")
